In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score
from keras.layers.convolutional import Conv1D

import numpy as np

import keras
import keras.layers as layers
import tensorflow as tf
import os
from nltk.tokenize import word_tokenize
from keras.layers import TextVectorization

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [90]:
data = pd.read_csv("processed_data.csv")
list(data.columns)
# drop unwanted features

data.iloc[:,5:].columns

# Drop all the column , keep only class , text_without_stopwords and title_without_stopwords
data = data.drop(data.iloc[:,5:].columns, axis=1)

data = data.drop(['title', 'text'],axis=1)
    
# ['title', 'text', 'text_without_stopwords', 'title_without_stopwords','syllables', 'polarity_category', 'overall_content', 'polarity'], axis=1)
#'Topic 1 Probability', 'Topic 2 Probability', 'Topic 3 Probbility' , 'Topic 4 Probability' ,'Topic 5 Probability',
#'title_word_count', 'title_sentence_count', 'title_average_word_length','title_punctuation_count', 'title_stopwords_count'  
# 'polarity_category_Neutral' , 'polarity_category_Positive'

In [91]:
#dataset is slightly imbalanced so we will perform upsampling to balance the dataset.
data['class'].value_counts()

0    21196
1    17462
Name: class, dtype: int64

In [92]:
list(data.columns)

['class', 'text_without_stopwords', 'title_without_stopwords']

In [93]:
data["overall_text"] = data["text_without_stopwords"] + " " + data["title_without_stopwords"]

data

,class,text_without_stopwords,title_without_stopwords,overall_text
0,1,donald trump wish americans happy new year lea...,donald trump sends out embarrassing new year’s...,donald trump wish americans happy new year lea...
1,1,house intelligence committee chairman devin nu...,drunk bragging trump staffer started russian c...,house intelligence committee chairman devin nu...
2,1,on friday revealed former milwaukee sheriff da...,sheriff david clarke becomes an internet joke ...,on friday revealed former milwaukee sheriff da...
3,1,on christmas day donald trump announced would ...,trump is so obsessed he even has obama’s name ...,on christmas day donald trump announced would ...
4,1,pope francis used annual christmas day message...,pope francis just called out donald trump duri...,pope francis used annual christmas day message...
...,...,...,...,...
38653,0,nato allies tuesday welcomed president donald ...,'fully committed' nato backs new us approach a...,nato allies tuesday welcomed president donald ...
38654,0,lexisnexis provider legal regulatory business ...,lexisnexis withdrew two products chinese market,lexisnexis provider legal regulatory business ...
38655,0,in shadow disused soviet-era factories minsk s...,minsk cultural hub becomes authorities,in shadow disused soviet-era factories minsk s...
38656,0,vatican secretary state cardinal pietro paroli...,vatican upbeat possibility pope francis visiti...,vatican secretary state cardinal pietro paroli...


### Prepare the data

In [94]:
#first split the dataset into training and test sets


x_train, x_test, y_train, y_test = train_test_split(data['overall_text'],data['class'],test_size=0.2,random_state = 4222)

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.25, random_state = 4222)


#balance x_train with oversampling
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')

# OverSampling only works on Dataframe, but current x_train and y_train are series
x_train, y_train = oversample.fit_resample(x_train.to_frame(), y_train.to_frame())


#check that train set is oversampled
y_train.value_counts()

class
0        12717
1        12717
dtype: int64

In [95]:
print(x_train)

                                            overall_text
0      venezuelans vote sunday nationwide mayoral pol...
1      the law order candidate broke law againdonald ...
2      the reason republican presidential frontrunner...
3      a senior israeli minister thursday declined co...
4      donald trump treated americans oppose enemy sa...
...                                                  ...
25429  randy johnson 21st century wire mirror mirror ...
25430  well first all politicians especially elected ...
25431  west virginia proud coal mining history rooted...
25432  truck drivers 100% fed up illegal muslim refug...
25433  at 8:20 friday morning first hopefully two bab...

[25434 rows x 1 columns]


In [96]:
# Find the number of maximum text 
print(data.overall_text.str.len().max())

40567


In [97]:
# TA example  not working for me. 

# Change dataframe back to Series
# # Model constants.

max_features = 10000
maxlen = 500 # Set a max length of the array, if not it will do an array of like [1,10000] , and if i would to run the LSTM, it will take 30 hours

raw_train_ds = tf.data.Dataset.from_tensor_slices(
    (tf.constant(x_train.squeeze().to_list()),
     tf.keras.utils.to_categorical(y_train.to_numpy() -1))
).batch(2048)

raw_test_ds = tf.data.Dataset.from_tensor_slices(
    (tf.constant(x_test.squeeze().to_list()),
     tf.keras.utils.to_categorical(y_test.to_numpy() -1))
).batch(2048)

raw_val_ds = tf.data.Dataset.from_tensor_slices(
    (tf.constant(x_validation.squeeze().to_list()),
     tf.keras.utils.to_categorical(y_validation.to_numpy() -1))
).batch(2048)

raw_train_ds

vectorize_layer = TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=42000, # Based on how many words there are in overall_text
)

text_ds = raw_train_ds.map(lambda x, y : x)

vectorize_layer.adapt(text_ds)

print(len(vectorize_layer.get_vocabulary()))
print(vectorize_layer.get_vocabulary()[:20])

def vectorize_text(text,label):
    return vectorize_layer(text),label

# Vectorize the Data
train_ds = raw_train_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU
train_ds = train_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)


10000
['', '[UNK]', 'trump', 'said', 'the', 'us', 'would', 'i', 'president', 'people', 'it', 'one', 'state', 'new', 'also', 'donald', 'house', 'republican', 'government', 'he']


In [98]:
for x, y in train_ds.take(1):
    print(x[0][:20])
    print(y.shape)

tf.Tensor(
[   1   75  258 3036    1  676 7608  264  340  634  221  143 1780    8
 6478 2427 9034  178  587 9254], shape=(20,), dtype=int64)
(2048, 1)


In [99]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train.squeeze())
tokenized_train = tokenizer.texts_to_sequences(x_train.squeeze())
x_train = pad_sequences(tokenized_train , maxlen=maxlen)

In [100]:
x_train

array([[   0,    0,    0, ...,   74, 3354, 2415],
       [   0,    0,    0, ...,   16, 9361,  958],
       [   0,    0,    0, ...,  129,  536, 1240],
       ...,
       [   0,    0,    0, ...,   61, 2170,   42],
       [   0,    0,    0, ...,  480,  838,   42],
       [   0,    0,    0, ...,  838, 3209, 9653]])

In [101]:
tokenized_test = tokenizer.texts_to_sequences(x_validation)
x_validation = pad_sequences(tokenized_test , maxlen=maxlen)

In [102]:
x_validation

array([[   0,    0,    0, ..., 1177,  655,  708],
       [   0,    0,    0, ..., 1027,  160,   19],
       [   0,    0,    0, ...,  597,    3, 1120],
       ...,
       [   0,    0,    0, ...,  106, 6430, 1034],
       [   0,    0,    0, ..., 1301,  453,  928],
       [   0,    0,    0, ...,  190, 4811,   70]])

In [103]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(tokenized_test , maxlen=maxlen)


In [104]:
x_test

array([[   0,    0,    0, ...,   40,  301,  121],
       [   0,    0,    0, ...,  802,   17,  337],
       [   0,    0,    0, ..., 4553, 9993,  950],
       ...,
       [   0,    0,    0, ...,  121,  893,  211],
       [   0,    0,    0, ..., 9927, 2736, 2663],
       [   0,    0,    0, ...,  302,   72,  805]])

## Build A model

In [105]:
max_features = 10000
batch_size = 256
epochs = 10
embed_size = 100

model = keras.models.Sequential()
model.add(layers.Embedding(max_features,embed_size))
model.add(layers.Dropout(0.5))

model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(layers.GlobalMaxPool1D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation="sigmoid",name="predictions"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])


## Train and evaluate the model

In [106]:
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 100)         1000000   
                                                                 
 dropout_10 (Dropout)        (None, None, 100)         0         
                                                                 
 conv1d_4 (Conv1D)           (None, None, 32)          9632      
                                                                 
 global_max_pooling1d_4 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dropout_11 (Dropout)        (None, 32)                0         
                                                      

In [107]:
print(x_train.shape)
print(x_test.shape)
print(x_validation.shape)

(25434, 500)
(7732, 500)
(7732, 500)


In [108]:
model.fit(x_train, y_train, validation_split=0.25, epochs=2)

Epoch 1/2
597/597 [==============================] - 25s 40ms/step - loss: 0.2109 - accuracy: 0.9097 - val_loss: 0.0437 - val_accuracy: 0.9833
Epoch 2/2
597/597 [==============================] - 24s 40ms/step - loss: 0.0358 - accuracy: 0.9887 - val_loss: 0.0263 - val_accuracy: 0.9903


In [109]:
model.fit(x_train, y_train,validation_data=(x_validation,y_validation),epochs=2)

Epoch 1/2
795/795 [==============================] - 32s 40ms/step - loss: 0.0184 - accuracy: 0.9944 - val_loss: 0.0250 - val_accuracy: 0.9922
Epoch 2/2
795/795 [==============================] - 31s 39ms/step - loss: 0.0074 - accuracy: 0.9977 - val_loss: 0.0226 - val_accuracy: 0.9929


In [110]:
# Prediction if the news is fake 
# Class 1 (Fake) if predicted prob >= 0.5, else class 0 (Real)

y_pred = (model.predict(x_test) >= 0.5).astype("int")


242/242 [==============================] - 1s 4ms/step


In [111]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4235
           1       0.99      0.99      0.99      3497

    accuracy                           0.99      7732
   macro avg       0.99      0.99      0.99      7732
weighted avg       0.99      0.99      0.99      7732



### Analysis after Training Model

In [112]:
print("Accuracy of the model on Training Data is - " , model.evaluate(x_train,y_train))
print("Accuracy of the model on Validation Data is - " , model.evaluate(x_validation,y_validation))
print("Accuracy of the model on Testing Data is - " , model.evaluate(x_test,y_test))


795/795 [==============================] - 4s 5ms/step - loss: 4.1196e-04 - accuracy: 1.0000
Accuracy of the model on Training Data is -  [0.000411961053032428, 0.9999606609344482]
242/242 [==============================] - 1s 4ms/step - loss: 0.0226 - accuracy: 0.9929
Accuracy of the model on Validation Data is -  [0.02255597896873951, 0.9928867220878601]
242/242 [==============================] - 1s 4ms/step - loss: 0.0232 - accuracy: 0.9933
Accuracy of the model on Testing Data is -  [0.023189663887023926, 0.9932746887207031]
